# Plan for an automated, multi-step analytical pipeline of the interface residues

__Current Implementation__: 
- Identify and retrieve the PDB entries 6M0J and 7Z0X.
- Perform structural superimposition of the two complexes to determine the overlap of the binding interfaces using PyMOL.
- Extract the interface residues in each complex using InterfaceResidues within PyMol
- Perform a comparative analysis of the interface residues of SARS-CoV-2 spike protein in the two complexes to identify important residues

In [24]:
import urllib.request
import pymol
import sys
import re
from typing import Dict, Tuple

# If you want to reproduce download InterfaceResidues.py and modify the following path. 
sys.path.append('/home/or22568/miniconda3/envs/bio-compchem/lib/python3.8/site-packages/pymol/')
from InterfaceResidues import interfaceResidues

In [2]:
pymol.finish_launching(['pymol', '-qc'])

### Identify and retrieve the PDB entries 6M0J and 7Z0X

In [12]:
prot1 = '6M0J'
prot2 = '7Z0X'

In [13]:
def Import_PDB() -> None:
    # SARS-CoV-2 spike protein bound with ACE2
    url = 'https://files.rcsb.org/download/6M0J.pdb'
    urllib.request.urlretrieve(url, '6M0J.pdb') # Save PDB as local file
    # Antibody THSC20.HVTR26
    url = 'https://files.rcsb.org/download/7Z0X.pdb'
    urllib.request.urlretrieve(url, '7Z0X.pdb') # Save PDB as local file

In [14]:
Import_PDB()

We save images of the loaded protein for visual inspection

In [15]:
def Save_Protein(name: str) -> None:
    # Save figures
    pymol.cmd.show("cartoon", name)
    pymol.cmd.png(name + ".png", width=800, height=600, dpi=300, ray=1)
    pymol.cmd.hide('everything', name)
        
def Load_Protein(name: str) -> None:
    # Load structures
    pymol.cmd.load(name +'.pdb', name)
    Save_Protein(name)
    
def Load_Proteins() -> None:
    Load_Protein(prot1) 
    Load_Protein(prot2)

In [16]:
Load_Proteins()

 Ray: render time: 0.93 sec. = 3854.0 frames/hour (0.93 sec. accum.).
 Ray: render time: 1.04 sec. = 3448.9 frames/hour (1.98 sec. accum.).


### Perform structural superimposition of the two complexes to determine the overlap of the binding interfaces using PyMOL.

In [17]:
# Here we align a pair of models. The second (prot2) is kept fix while the first one (prot1) is rotated
def Align_Structures() -> None:
    pymol.cmd.show('cartoon', prot1)
    pymol.cmd.show('cartoon', prot2)
    pymol.cmd.align(prot1, prot2)
    # We save the figure to for visual inspection
    pymol.cmd.png("aligned_6M0J.png", width=800, height=600, dpi=300, ray=1)
    return 

In [18]:
Align_Structures()

 Ray: render time: 2.32 sec. = 1554.2 frames/hour (4.29 sec. accum.).


To determine which chain corresponds to the SARS-CoV-2 spike protein and which ones correspond to the antibody in the PDB entries provided.

In [19]:
all_objs = pymol.cmd.get_object_list()
for obj in all_objs:
    chains = pymol.cmd.get_chains(obj)
    print(obj, chains)

6M0J ['A', 'E']
7Z0X ['A', 'H', 'L', 'R']


### Extract the interface residues in each complex using InterfaceResidues within PyMol

In [30]:
# interfaceResidues works with only two chains! 
# If a complex has more than two chains a pariwise comparison is necessary. The output will be the interface 
# between each pair. 
def Identify_Binding_Interfaces(obj: str, chain1: str, chain2: str):
    myInterfaceResidues = interfaceResidues(obj, cA=chain1, cB=chain2, cutoff=0.75, selName="interface")
    # get_fastastr, get the fasta format of the selected residues ("interface")
    list_fasta = pymol.cmd.get_fastastr(selection='interface', state=-1, quiet=1)
    # In the figure we save the residues with sticks
    pymol.cmd.show("sticks", "interface")
    pymol.cmd.png(obj + "_interface.png", width=800, height=600, dpi=300, ray=1)
    return myInterfaceResidues, list_fasta

In [31]:
# To have a visual inspection we manipulate the objects in the PyMol by showing and hiding the needed one
pymol.cmd.show("cartoon", prot1)    
pymol.cmd.hide('everything', prot2)
# Get the binding interface of 6M0J
binding1_res, binding1_fasta = Identify_Binding_Interfaces(prot1, "c. A", "c. E")

pymol.cmd.hide('everything', prot1)
pymol.cmd.show("cartoon", prot2)
# Get the binding interface of 7Z0X. Here automation can be improved. 
# Here we compare spike with heavy chain and after spike with light chain. 
binding2_res, binding2_fasta = Identify_Binding_Interfaces(prot2, "c. H", "c. R")
binding3_res, binding3_fasta = Identify_Binding_Interfaces(prot2, "c. L", "c. R")

 Ray: render time: 1.13 sec. = 3176.3 frames/hour (16.10 sec. accum.).
 Ray: render time: 0.78 sec. = 4604.0 frames/hour (16.88 sec. accum.).
 Ray: render time: 0.80 sec. = 4497.3 frames/hour (17.68 sec. accum.).


### Perform a comparative analysis of the interface residues of SARS-CoV-2 spike protein in the two complexes to identify important residues

In [36]:
# I need this help function to count the number of residues for each interfaces
def Count_Element(arr: list, element: str) -> int:
    count = 0
    for item in arr:
        if type(item) == list or type(item) == tuple:
            count += Count_Element(item, element)
        elif item == element:
            count += 1
    return count

In [37]:
# This help function allow me to get the residues of the interfaces and extract the fasta format from each residue.
# I save the informations in two dictionaries interface_spike and interface_antibody. 
# Each of this dictionary has the residue number, the fasta format and the distance from the interface
def Define_Interfaces(binding: list, fasta: list, interface_spike: Dict, interface_antibody: Dict) -> Tuple[Dict]:
    # Count residues for spike and antibody
    n_antibody = Count_Element(binding, 'chA') 
    n_spike = Count_Element(binding, 'chB')
    print("Number residues interface antibody: ", n_antibody)
    print("Number residues interface spike: ", n_spike)
    
    index = 0 
    for ele in binding:
        try:
            res, dist = int(ele[1]), float(ele[2])
        except:
            res = int(re.sub("[^0-9]", "", ele[1]))
        if ele[0] == 'chA':
            label = fasta.split()[1][index]
            interface_antibody[res] = [label, f"{dist:5.2f}"]
            index += 1
        elif ele[0] != 'chA':
            label = fasta.split()[3][index - n_antibody]
            interface_spike[res] = [label, f"{dist:5.2f}"]
            index += 1
    return interface_spike, interface_antibody

In [38]:
# I need this help function to print the sequence of amino acids
def Print(dictionary: Dict) -> None:
    value_min, value_max = int(min(dictionary.keys())), int(max(dictionary.keys()))
    string = ''
    # I leave 5 residues empty to have a nice print of the sequence
    for i in range(value_min - 5, value_max + 5):
        if i in list(dictionary.keys()):
            string += dictionary[i][0]
        else:
            string += '.'
    print(string)

In [43]:
# I need this help function to print the most important residuals. These are excellent candidates for further mutations
def Print_Important_Residues(dictionary: Dict) -> Dict:
    important_residues = {}
    for key, values in dictionary.items():
        #print(values)
        if float(values[1]) > 4.0: 
            important_residues[key] = [values]
            print(f"Reside: {key:<5}, Amino Acid: {values[0]}, Distance (dASA): {values[1]}")
    #return important_residues

In this section, we display the output of our analysis, which includes the interface sequences of the spike protein and antibody. The interface amino acids are presented in the FASTA format. Additionally, we highlight the closest and most significant amino acids.

In [44]:
interface_spike, interface_antibody = Define_Interfaces(binding1_res, binding1_fasta, {}, {})
print("\nSeq. for Spike interface M60J")
Print(interface_spike)
print("\nSeq. for Anitbody M60J")
Print(interface_antibody)
print("\nRelevant Residues for the antibody interface M60J:\n")
Print_Important_Residues(interface_antibody)

Number residues interface antibody:  25
Number residues interface spike:  22

Seq. for Spike interface M60J
.....K...........................VG..Y...Y.LF................Y.AG.......E.FN.Y...Q..G.Q.TNGV.Y....

Seq. for Anitbody M60J
.....S....Q..TF.DK..HE.ED..YQ..L.................................L..MY.................................................................................................................................................................................................................................................QG...N......................KGD.R............................A......R....

Relevant Residues for the antibody interface M60J:

Reside: 19   , Amino Acid: S, Distance (dASA):  8.77
Reside: 28   , Amino Acid: F, Distance (dASA):  5.10
Reside: 31   , Amino Acid: K, Distance (dASA):  5.02
Reside: 41   , Amino Acid: Y, Distance (dASA):  4.59
Reside: 42   , Amino Acid: Q, Distance (dASA): 35.01
Reside: 79   , Amino Acid: L, Distance (dASA): 20.68
Reside: 82   

In [45]:
interface_spike, interface_antibody1 = Define_Interfaces(binding2_res, binding2_fasta, {}, {})
interface_spike, interface_antibody2 = Define_Interfaces(binding3_res, binding3_fasta, interface_spike, {})
print("\nSeq. for Spike interface 7Z0X")
Print(interface_spike)
print("\nSeq. for Anitbody Heavy chain 7Z0X")
Print(interface_antibody1)
print("\nSeq. for Anitbody Light chain 7Z0X")
Print(interface_antibody2)
print("\nRelevant Residues for the antibody interface Heavy chain 7Z0X:\n")
Print_Important_Residues(interface_antibody1)
print("\nRelevant Residues for the antibody interface Heavy chain 7Z0X:\n")
Print_Important_Residues(interface_antibody2)

Number residues interface antibody:  13
Number residues interface spike:  14
Number residues interface antibody:  6
Number residues interface spike:  6

Seq. for Spike interface 7Z0X
.....F................Y.AGSTP.N.VEGFNCY...Q....

Seq. for Anitbody Heavy chain 7Z0X
.....Y..................YSG.S.Y....................................LVGALG....

Seq. for Anitbody Light chain 7Z0X
.....Y.L..........................................................Y..SSW....

Relevant Residues for the antibody interface Heavy chain 7Z0X:

Reside: 95   , Amino Acid: L, Distance (dASA):  5.21
Reside: 98   , Amino Acid: A, Distance (dASA):  8.92
Reside: 99   , Amino Acid: L, Distance (dASA): 10.62
Reside: 100  , Amino Acid: G, Distance (dASA): 10.62

Relevant Residues for the antibody interface Heavy chain 7Z0X:

Reside: 30   , Amino Acid: Y, Distance (dASA):  5.16
Reside: 95   , Amino Acid: S, Distance (dASA): 18.56
